# <center>三 LangGraph应用程序运维监控

&emsp;&emsp;大模型具有不确定性，尤其是构建复杂`AI Agent`应用程序中，中间会涉及非常多的子步骤，如果想要了解每一步的运行状态和结果，一方面可以通过`Debug`来进行实时控制，而另一方面可以借助一些工具来观察和调试中间的交互流程。`Langsmith`就是这样一个工具平台， 由​​ `LangChain` 和 `LangGraph` 背后的团队创建，**主要作用是：为基于大语言模型构建的应用程序提供全面的监控、调试和可观察性。提供强大的跟踪、日志记录和实时分析功能。**

> LangSmith：https://smith.langchain.com/

&emsp;&emsp;通常，对于一个项目而言，可以是单个应用程序或服务。该项目将包含多个跟踪，每个跟踪都是运行的集合 - 一个运行代表应用程序中的一个基本操作，例如对 OpenAI 的 API 调用，或检索操作。如下图所示：

<div align=center><img src="../pic/lesson03/1.png" width=100%></div>

- **Project (项目)** 蓝色方块代表整个项目，可能是一个单独的应用程序或服务。
- **Traces (轨迹)** 绿色方块代表项目在不同条件或配置下的执行路径。每个轨迹可以是一个特定的用户会话、一个功能的执行，或者应用在特定输入下的行为。
- **Runs (运行)** 每个轨迹下的黄色方块表示特定轨迹的单次执行。这些是执行的实例，每个实例都是轨迹在特定条件下的实际运行。
- **Feedback, Tags, Metadata (反馈、标签、元数据)** 这部分显示了系统如何利用用户或自动化工具生成的反馈、标签和元数据来增强轨迹的管理和过滤。反馈可以用于改进未来的运行，标签和元数据可用于分类和筛选特定的轨迹或运行，以便在LangSmith的用户界面中更容易地管理和审查

## 1.1 注册账户

> LangSmith：https://smith.langchain.com/

## 1.2 使用

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

import getpass
import os

key="sk-kDWtZI5R2tOWLYiREfzzsD7Z3XEsEH5n6SClKqZ4Lxr5GcM"
base_url="https://chatapi.littlewheat.com/v1"

In [22]:

# 设置环境变量
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"
#os.environ["LANGCHAIN_PROJECT"] = "chat"
#os.environ["LANGCHAIN_ENDPOINT"]= "https://api.smith.langchain.com"

# 验证环境变量是否设置成功
print(os.getenv("LANGCHAIN_TRACING_V2"))
print(os.getenv("LANGCHAIN_API_KEY"))
print(os.getenv("LANGCHAIN_PROJECT"))

true
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
chat


In [23]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
import getpass
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)



llm = ChatOpenAI(model="gpt-4o", api_key=key,base_url=base_url,temperature=0,)

def chatbot(state: State):
    # print(state)
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [24]:
def stream_graph_updates(user_input: str):  
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("模型回复:", value["messages"][-1].content)


while True:
    try:
        user_input = input("用户提问: ")
        if user_input.lower() in ["退出"]:
            print("下次再见！")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available  
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

用户提问:  你好


模型回复: 你好！有什么我可以帮忙的吗？


用户提问:  介绍一下自己


模型回复: 你好！我是一个大型语言模型助手，由OpenAI训练开发。我可以帮助回答问题、提供信息、撰写文章以及协助许多其他任务。我特别擅长处理和理解各种主题的自然语言。如果你有任何问题或需要帮助的地方，随时可以问我！


用户提问:  退出


下次再见！
